In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
df = pd.read_csv('../../data/csl/CSL_tytl_PI.csv', index_col=0)
#corr_df = pd.read_csv('../../data/csl/Features/PI_Tl_Del_noSiteCorr.csv', index_col=0, header=None, delimiter='\t')
#corr_df = pd.read_csv('../../data/csl/Features/PI_Tl_Del_noSite_abCont_u50.csv', index_col=0, header=None, delimiter='\t')
corr_df = pd.read_csv('../../data/csl/Features/PI_Tl_Del_noSite_abCont_ALL.csv', index_col=0, header=None, delimiter='\t')
corr_vars = list(corr_df.index.values)

# This version takes smaller samples, close to the size of the individual Sites, to verify that it isn't the sample size that is causing the Sites to perform worse.

In [3]:
%%time
from mwb_bootstrap import bootstrap_stat_samp
from sklearn.ensemble import GradientBoostingClassifier

X = df.drop('trans_loss', axis=1, inplace=False)
X = X[corr_vars]
y = df['trans_loss'].values
    
clf = GradientBoostingClassifier(n_estimators=70, criterion="friedman_mse",max_depth=11, 
                          min_samples_leaf=50, min_samples_split=900,max_leaf_nodes=None,
                          max_features=12,subsample=0.9, learning_rate=0.1,random_state=7)

stats_df, feats_df, X_train = bootstrap_stat_samp(X, y, clf, samp_size=30000, test_size=0.30, 
                                                 sample_weights=True, nsamples=100, under=False)
#stats_df, feats_df, X_train = bootstrap_stat_samp(X, y, clf, samp_size=10, test_size=0.25, 
#                                                 sample_weights=True, nsamples=3, under=False)
stats_df.loc['mean'] = stats_df.mean()
stats_df.loc['mean',:]

(no random(seed) call) Mean y_pos = 0.05570533333333334
CPU times: user 3min 1s, sys: 372 ms, total: 3min 1s
Wall time: 3min 1s


recall     0.799126
prec       0.138249
MCC        0.247977
PR_AUC     0.198236
roc_auc    0.822942
Name: mean, dtype: float64

In [4]:
feats_df['mean'] = feats_df.mean(axis=1)

In [5]:
feats_df['median'] = feats_df.median(axis=1)
feats_df['std'] = feats_df.std(axis=1)

In [6]:
sort_feat = feats_df.sort_values(by='mean', ascending=False)

In [7]:
sort_feat[['mean','median','std']].head(20)

,mean,median,std
Delmode,0.151710,0.149064,0.018629
Intratocolytix,0.050464,0.049665,0.011605
Inoxy_incrdose,0.048272,0.049731,0.010344
CS_FTP,0.027891,0.027891,0.006692
Analgesia,0.026837,0.026406,0.003996
Insurance,0.025168,0.023555,0.007008
Dilat_lst,0.022645,0.022294,0.005902
Admconsistency,0.022592,0.021423,0.005128
ROMmeth,0.021613,0.021558,0.005513
Hxanemia,0.020881,0.020643,0.005112
